In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
from matplotlib import cm, colors
from matplotlib.patches import Circle, Arc, Polygon, Wedge

def od_hit_pattern(p_areas, title="", centering=True, verbose=True, z=-1, theta=-1):
    
    """
    RMG April 2023
    
    Inputs
      p_areas: 1D np array of size 120, areas in correct PMT ordering
      title: string, title shown on plot
      centering: bool, center on PMT with max light
      verbose: bool, show ch areas in plot
      z, theta: floats, position to be shown in plot
    Outputs
      fig, ax: mpl figure and axes of plot
    """
    
    # Plotting parameters
    fig, ax = pl.subplots(figsize=(35,10))
    pl.rcParams.update({'font.size': 18})
    pl.rcParams.update({'font.family': 'serif'})
    cmap = cm.get_cmap("Reds")
    norm = colors.Normalize(0, max(p_areas))
    cb = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, pad=0.01)
    cb.set_label("Ch pulse area [phd]")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    pl.xticks([])
    pl.yticks([])
    pl.xlim(-10,200)
    pl.ylim(-10,60)
    pl.title(title)
    
    column_max = np.argmax(p_areas)%20 # where to center the PMTs
    if not centering: column_max = 0 #9
    
    # Draw HV and DD first
    ax.add_patch(Circle(xy=(((9-column_max+4.5)%20)*10, 1.6*10),radius=4,facecolor="black",alpha=0.1)) # HV
    pl.annotate("HV",xy=(((9-column_max+4.5)%20)*10-1.6, 1.6*10-0.7),alpha=0.6,fontsize=15) # HV
    ax.add_patch(Circle(xy=(((9-column_max+9.5)%20)*10, 3.3*10),radius=2,facecolor="black",alpha=0.1)) # DD straight
    pl.annotate("DD",xy=(((9-column_max+9.5)%20)*10-1.4, 3.3*10-0.4),alpha=0.6,fontsize=15) # DD straight
    ax.add_patch(Wedge(center=(((9-column_max+14.5)%20)*10, 3*10),r=2,theta1=0,theta2=180,facecolor="black",alpha=0.1)) # DD angled
    ax.add_patch(Polygon(xy=[(((9-column_max+14.5)%20)*10-2, 3*10),(((9-column_max+14.5)%20)*10+2, 3*10),(((9-column_max+14.5)%20)*10, 3*10-18)],
                         closed=True,facecolor="black",alpha=0.1,edgecolor="none")) # DD angled
    pl.annotate("DD",xy=(((9-column_max+14.5)%20)*10-1.4, 3*10-0.4-1.5),alpha=0.6,fontsize=15) # DD angled
    
    # Draw PMTs
    for i in range(6):
        for j in range(20):
            ax.add_patch(Circle(xy=(j*10,(5-i)*10), radius=2,alpha=1,edgecolor="black",facecolor="none")) # edges
            this_pmt = i*20 + (column_max-9+j)%20 # pmt indexing
            ax.add_patch(Circle(xy=(j*10,(5-i)*10), radius=1.9,edgecolor="none",facecolor=cmap(norm(p_areas[this_pmt])) )) # faces
            if verbose: pl.annotate(round(p_areas[this_pmt],2),xy=(j*10-1.5,(5-i)*10-3.5),fontsize=12) # show ch areas
            
    # Row and column numbers
    for i in range(6): pl.annotate(i*20+800,xy=(-9,(5-i)*10-0.7))
    for j in range(20): pl.annotate((column_max-9+j)%20 ,xy=(j*10-1,55))
    pl.axvline(-3.5,color="black")
    pl.axhline(53.5,color="black")
    
    # Draw reconstructed position
    if z != -1 and theta != -1:
        pl.plot( (9-column_max + 20*theta/(2*np.pi))*10, (5-(z-249.5)/(-70))*10  , "g*", markersize=30, alpha=0.5  )
    
    pl.rcParams.update({'font.size': 18})
    pl.show()
    
    return fig, ax